### Imports

In [10]:
import os
import numpy as np
import pandas as pd
from pykalman import KalmanFilter

### Inputs

In [11]:
fusion_output_df = pd.read_csv("../outputs/fusion_output.csv")
fusion_output_class = fusion_output_df['class'].values
fusion_output_confidence = fusion_output_df['confidence'].values

In [12]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df[["rssi", "etx", "delay", "busy_fraction"]].values

In [13]:
radnet_input_df = pd.read_csv("../inputs/radnet_input.csv")
radnet_input_efsr_edge = radnet_input_df['efsr_edge'].values

### Kalman Filter

In [14]:
confidence_fusion = np.where(fusion_output_class == 1,
    fusion_output_confidence,
    1.0 - fusion_output_confidence
)

In [15]:
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    initial_state_mean=confidence_fusion[0],
    initial_state_covariance=1.0,
    observation_covariance=1e-2,  # R
    transition_covariance=2e-2    # Q
)

In [16]:
kalman, _ = kf.filter(confidence_fusion)
kalman = kalman.flatten()

cls = (kalman >= 0.5).astype(int)
conf = np.where(cls == 1, kalman, 1.0 - kalman)

In [17]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'EFSR':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*75}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{radnet_input_efsr_edge[i]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[int(cls[i])]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   EFSR   Classe   Status   Confiança 
---------------------------------------------------------------------------
-60    1.3   5      0.45   0.92   1        Bom      0.69      
-67    1.7   3      0.26   0.91   1        Bom      0.68      
-74    4.1   19     0.52   0.43   0        Ruim     0.61      
-64    1.3   15     0.50   0.88   1        Bom      0.58      
-60    3.4   70     0.78   0.50   0        Ruim     0.71      
-70    6.7   146    0.85   0.38   0        Ruim     0.78      
-63    4.9   35     0.83   0.41   0        Ruim     0.79      
-59    2.6   7      0.42   0.44   0        Ruim     0.60      
-76    5.6   56     0.69   0.33   0        Ruim     0.76      
-61    1.5   13     0.22   0.95   1        Bom      0.61      
-52    9.7   31     0.75   0.88   1        Bom      0.52      
-83    7.9   119    0.29   0.33   0        Ruim     0.73      
-93    12.5  47     0.71   0.39   0        Ruim     0.79      
-65    2.8   26     0.70   0.93   0       

### Save

In [18]:
kalman_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
kalman_output.to_csv("../outputs/kalman_output.csv", index=False)

print("Filtro de Kalman salvo em ../outputs/kalman_output.csv")

Filtro de Kalman salvo em ../outputs/kalman_output.csv
